In [3]:
import pandas as pd
import numpy as np
from pydataset import data

In [2]:
import pyspark
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, concat_ws, lit
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# 1
Create a spark data frame that contains your favorite programming languages.

    The name of the column should be language
    View the schema of the dataframe
    Output the shape of the dataframe
    Show the first 5 records in the dataframe

In [ ]:
#mas tarde

# 2
Load the mpg dataset as a spark dataframe.

    Create 1 column of output that contains a message like the one below:


    The 1999 audi a4 has a 4 cylinder engine.
    For each vehicle.

    Transform the trans column so that it only contains either manual or auto.

In [4]:
mpg = spark.createDataFrame(data("mpg"))

In [5]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



# 3
Load the tips dataset as a spark dataframe.

    What percentage of observations are smokers?
    Create a column that contains the tip percentage
    Calculate the average tip percentage for each combination of sex and smoker.

# 4
Use the seattle weather dataset referenced in the lesson to answer the questions below.

    Convert the temperatures to farenheight.
    Which month has the most rain, on average?
    Which year was the windiest?
    What is the most frequent type of weather in January?
    What is the average high and low tempurature on sunny days in July in 2013 and 2014?
    What percentage of days were rainy in q3 of 2015?
    For each year, find what percentage of days it rained (had non-zero precipitation).